Project 1 - Terrence Cummings
- Crime, Schools, and Home Values in Minneapolis

Step 1 - Housing data
- Read in housing data from "Open Minneapolis" website using APIs (currently hit 2000 limit. Need to fix.)
- Store in dataframe
- Cleanup data
- Convert x, y map coordinates to lat/lng

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import scipy.stats as st
import pandas as pd
import numpy as np
import geopandas as gpd
import requests
import time
from scipy.stats import linregress
import json
from pprint import pprint
from datetime import datetime


In [14]:
#Initialize lists to hold key home sales data
sale_id =[]
sale_date = []
formatted_address = []
land_sale = []
community_cd = []
community_desc = []
nbhd_cd = []
nbhd_desc = []
ward = []
proptype_cd = []
proptype_desc = []
adj_sale_price = []
gross_sale_price = []
downpayment = []
x_coord = []
y_coord = []

#Initialize API record offset because of 2000 record limit per API call
resultOffset_num = 0

#Base URL for the Open Minnesota database of home sales in Minneapolis
base_url = 'https://services.arcgis.com/afSMGVsC7QlRK1kZ/arcgis/rest/services/Property_Sales_2015_to_2019/FeatureServer/0/query?where=1%3D1&outFields=*&outSR=4326&f=json'

#Make successive API call to grab 2000 records each time. Total records is 41,053
while resultOffset_num<=50000:

#Increment the record offset to get the next batch of records
    resultOffset = str(resultOffset_num)
    resultRecordCount = str(2000)
    resultOffset_num = resultOffset_num+2000

#Dyamically adjust the offset parameter of the API call and create the next target URL
    pagination_url = f'&resultOffset={resultOffset}&resultRecordCount={resultRecordCount}'
    target_url = f'{base_url}{pagination_url}'

#Make the API call and store data
    home_sales_data = requests.get(target_url).json()


#Set the number of home sales in the data for looping in filling the lists of key data
    num_houses=len(home_sales_data['features'])

#Fill lists of key data
    for house in range(0, num_houses):
        sale_id.append(home_sales_data['features'][house]['attributes']['SALE_ID'])
        sale_date.append(home_sales_data['features'][house]['attributes']['SALE_DATE'])
        formatted_address.append(home_sales_data['features'][house]['attributes']['FORMATED_ADDRESS'])
        land_sale.append(home_sales_data['features'][house]['attributes']['LAND_SALE'])
        community_cd.append(home_sales_data['features'][house]['attributes']['COMMUNITY_CD'])
        community_desc.append(home_sales_data['features'][house]['attributes']['COMMUNITY_DESC'])
        nbhd_cd.append(home_sales_data['features'][house]['attributes']['NBHD_CD'])
        nbhd_desc.append(home_sales_data['features'][house]['attributes']['NBHD_DESC'])
        ward.append(home_sales_data['features'][house]['attributes']['WARD'])
        proptype_cd.append(home_sales_data['features'][house]['attributes']['PROPTYPE_CD'])
        proptype_desc.append(home_sales_data['features'][house]['attributes']['PROPTYPE_DESC'])
        adj_sale_price.append(home_sales_data['features'][house]['attributes']['ADJ_SALE_PRICE'])
        gross_sale_price.append(home_sales_data['features'][house]['attributes']['GROSS_SALE_PRICE'])
        downpayment.append(home_sales_data['features'][house]['attributes']['DOWNPAYMENT'])
        x_coord.append(home_sales_data['features'][house]['attributes']['X'])
        y_coord.append(home_sales_data['features'][house]['attributes']['Y'])


In [28]:
#Create df of all MSP home sales data
msp_home_sales_df = pd.DataFrame(zip(sale_id, sale_date, formatted_address, land_sale, community_cd, community_desc, nbhd_cd, nbhd_desc, ward, proptype_cd, proptype_desc, adj_sale_price, gross_sale_price, downpayment, x_coord, y_coord), columns = ['sale_id', 'sale_date', 'formatted_address', 'land_sale', 'community_cd', 'community_desc', 'nbhd_cd', 'nbhd_desc', 'ward', 'proptype_cd', 'proptype_desc', 'adj_sale_price', 'gross_sale_price', 'downpayment', 'x_coord', 'y_coord'])

msp_home_sales_df.set_index('sale_id', inplace=True)

#Create a df of the community codes and descriptions
community_df = pd.DataFrame(zip(community_cd, community_desc), columns=['community_cd', 'community_desc'])
community_df = community_df.drop_duplicates()
community_df = community_df.sort_values(['community_cd'])
community_df.set_index('community_cd', inplace = True)

#Create a df of the neighborhood codes and descriptions
nbhd_df = pd.DataFrame(zip(nbhd_cd, nbhd_desc), columns=['nbhd_cd', 'nbhd_desc'])
nbhd_df = nbhd_df.drop_duplicates()
nbhd_df = nbhd_df.sort_values(['nbhd_cd'])
nbhd_df.set_index('nbhd_cd', inplace = True)

#Create a df of the property type codes and descriptions
proptype_df = pd.DataFrame(zip(proptype_cd, proptype_desc), columns=['proptype_cd', 'proptype_desc'])
proptype_df = proptype_df.drop_duplicates()
proptype_df = proptype_df.sort_values(['proptype_cd'])
proptype_df.set_index('proptype_cd', inplace = True)



In [84]:
#Clean up df by eliminating very low sale price
clean_msp_home_sales_df = msp_home_sales_df.loc[msp_home_sales_df['adj_sale_price']>10000]

#Clean up df by eliminating very high sale price
clean_msp_home_sales_df = clean_msp_home_sales_df.loc[clean_msp_home_sales_df['adj_sale_price']<5000000]

#Clean up by eliminating non-residential and odd property types
proptype_allowed = ['R', 'X', 'DB', 'Y', 'RZ' 'RM', 'RL', 'TP']
clean_msp_home_sales_df = clean_msp_home_sales_df[clean_msp_home_sales_df['proptype_cd'].isin(proptype_allowed)]

#Clean up by eliminating land-only sales
clean_msp_home_sales_df = clean_msp_home_sales_df[clean_msp_home_sales_df['land_sale']=='NO']

#Delete unneeded column
del clean_msp_home_sales_df['land_sale']

#Format the sale date
clean_msp_home_sales_df['sale_date']= pd.to_datetime(clean_msp_home_sales_df['sale_date'], unit = 'ms')

clean_msp_home_sales_df


,sale_date,formatted_address,community_cd,community_desc,nbhd_cd,nbhd_desc,ward,proptype_cd,proptype_desc,adj_sale_price,gross_sale_price,downpayment,x_coord,y_coord
sale_id,,,,,,,,,,,,,,
304279,2015-01-09 06:00:00,3542 Valley St,2,NORTHEAST,11,COLUMBIA,1.0,R,RESIDENTIAL,145985,150500,7525,533346.190261,188201.516808
303578,2015-01-09 06:00:00,301 Oak Grove St #506,4,CENTRAL,31,LORING PARK,7.0,X,CONDOMINIUM,241000,241000,0,525967.100955,163990.980501
304190,2015-01-09 06:00:00,3136 West Bde Maka Ska Blvd W #206,6,CALHOUN-ISLE,49,WEST CALHOUN,13.0,X,CONDOMINIUM,20000,20000,0,516063.000000,156554.000000
303562,2015-01-09 06:00:00,730 4TH St N #601,4,CENTRAL,86,NORTHLOOP,3.0,X,CONDOMINIUM,320000,320000,0,526709.800000,171382.570000
304311,2015-01-09 06:00:00,3439 Polk St,2,NORTHEAST,12,WAITE PARK,1.0,R,RESIDENTIAL,153500,160000,18741,535920.652943,187323.980831
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349104,2019-12-11 06:00:00,3307 Oliver Ave N,1,CAMDEN,6,FOLWELL,4.0,R,RESIDENTIAL,201760,208000,3768,519653.507324,181750.950923
349106,2019-12-11 06:00:00,3305 32ND Ave S,8,LONGFELLOW,61,LONGFELLOW,2.0,DB,DOUBLE BUNGALOW,330000,330000,330000,540924.461117,155372.733934
348057,2019-12-11 06:00:00,1240 2ND St S #1027,4,CENTRAL,88,DOWNTOWN EAST,3.0,X,CONDOMINIUM,495803,495803,495803,534741.000000,167665.000000


In [87]:
clean_msp_home_sales_df.describe()

,community_cd,nbhd_cd,ward,adj_sale_price,gross_sale_price,downpayment,x_coord,y_coord
count,36265.000000,36265.000000,36265.000000,3.626500e+04,3.626500e+04,3.626500e+04,36264.000000,36264.000000
mean,5.814863,48.658293,7.154088,2.816078e+05,2.837968e+05,7.271988e+04,528977.912775,162467.001277
std,3.170320,27.336689,3.982461,2.246991e+05,2.247539e+05,1.229446e+06,8807.071446,15852.808844
min,1.000000,0.000000,1.000000,1.004200e+04,1.004200e+04,0.000000e+00,933.000000,136397.659001
25%,3.000000,25.000000,4.000000,1.606500e+05,1.640000e+05,2.196000e+03,522129.450866,149120.192232
50%,6.000000,54.000000,7.000000,2.328000e+05,2.350000e+05,1.700000e+04,528549.028961,160382.875405
75%,9.000000,71.000000,11.000000,3.305000e+05,3.350000e+05,7.000000e+04,535477.902553,176490.941854
max,11.000000,92.000000,13.000000,4.800000e+06,4.800000e+06,2.325093e+08,547403.088448,195519.000000
